In [ ]:
from openai import OpenAI
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from scipy.spatial.distance import cosine
import plotly.express as px
from typing import Iterable, List, Tuple, Dict, Optional
import os


# Helpers

def get_embeddings(
    client: OpenAI,
    texts: Iterable[str],
    model: str = "text-embedding-3-large",
    batch_size: int = 256
) -> List[List[float]]:
    """
    Batches text through OpenAI embeddings endpoint and returns a list of vectors.
    """
    embeddings: List[List[float]] = []
    batch: List[str] = []

    def _flush(batch_texts: List[str]):
        if not batch_texts:
            return []
        resp = client.embeddings.create(model=model, input=batch_texts)
        # ensure order is preserved 1:1 with inputs
        return [d.embedding for d in resp.data]

    for t in texts:
        batch.append(str(t))
        if len(batch) >= batch_size:
            embeddings.extend(_flush(batch))
            batch = []
    # flush remainder
    if batch:
        embeddings.extend(_flush(batch))

    return embeddings


def _encode_labels(series: pd.Series, mapping: Dict[str, int]) -> np.ndarray:
    return series.map(mapping).to_numpy()


def _gender_to_binary(g: pd.Series) -> pd.Series:
    """
    Map gender labels to binary (female=1, male=0); tolerant to 'girls/women' vs 'boys/men'.
    """
    female_set = {"girl", "girls", "woman", "women", "female", "f"}
    male_set = {"boy", "boys", "man", "men", "male", "m"}
    return g.str.lower().map(lambda x: 1 if x in female_set else (0 if x in male_set else np.nan))


def _mean_vec(vectors: np.ndarray) -> np.ndarray:
    return np.asarray(vectors, dtype=float).mean(axis=0)


# Main pipeline

def run_bias_pipeline(
    csv_path: str,
    text_col: str = "text",
    sentiment_col: str = "sentiment",
    gender_col: str = "gender",
    split_col: str = "dataSet",
    embedding_model: str = "text-embedding-3-large",
    pca_components: int = 2,
    sentiment_labels: Tuple[str, str] = ("negative", "positive"),
    plot: bool = True,
    save_embedded_csv: Optional[str] = "data_set_with_embeddings.csv",
    save_pickle: Optional[str] = "data_set_with_embeddings.pickle",
    permutation_runs: int = 1000,
    random_state: int = 42,
    api_key: Optional[str] = None
):
    """
    End-to-end bias pipeline.
    """
    # --- Load API key safely ---
    if api_key is None:
        api_key = os.getenv("OPENAI_API_KEY")

    if api_key is None:
        raise ValueError(
            "No API key provided. Set OPENAI_API_KEY environment variable.\n"
            "Example:\n"
            'export OPENAI_API_KEY="your_key_here"\n'
        )

    rng = np.random.default_rng(random_state)
    client = OpenAI(api_key=api_key)

    # --- Load dataset ---
    df = pd.read_csv(csv_path)
    needed = {text_col, sentiment_col, gender_col}
    missing = needed - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns in CSV: {missing}")

    # --- Embeddings ---
    df = df.copy()
    df["embedding"] = get_embeddings(client, df[text_col].tolist(), model=embedding_model)
    emb = np.stack(df["embedding"].to_numpy())

    # --- Save with embeddings ---
    if save_embedded_csv:
        df.to_csv(save_embedded_csv, index=False)
    if save_pickle:
        df.to_pickle(save_pickle)

    # --- PCA (2D) ---
    if pca_components and pca_components >= 2:
        pca = PCA(n_components=2, random_state=random_state)
        df[["pca_x", "pca_y"]] = pca.fit_transform(emb)

        if plot:
            fig_pca = px.scatter(
                df,
                x="pca_x",
                y="pca_y",
                color=gender_col,
                symbol=sentiment_col,
                title="Embeddings (PCA 2D)"
            )
            fig_pca.show()

    # --- Train/test split ---
    if split_col in df.columns:
        train_df = df[df[split_col].str.lower() == "train"].copy()
        test_df  = df[df[split_col].str.lower() == "test"].copy()
        if train_df.empty or test_df.empty:
            df = df.sample(frac=1.0, random_state=random_state)
            mask = np.zeros(len(df), dtype=bool)
            mask[: int(2 * len(df) / 3)] = True
            train_df, test_df = df[mask].copy(), df[~mask].copy()
    else:
        df = df.sample(frac=1.0, random_state=random_state)
        mask = np.zeros(len(df), dtype=bool)
        mask[: int(2 * len(df) / 3)] = True
        train_df, test_df = df[mask].copy(), df[~mask].copy()

    # --- Sentiment classifier ---
    neg_label, pos_label = sentiment_labels
    y_train_sent = train_df[sentiment_col].str.lower().map({neg_label: 0, pos_label: 1}).to_numpy()
    y_test_sent  = test_df[sentiment_col].str.lower().map({neg_label: 0, pos_label: 1}).to_numpy()

    X_train = np.stack(train_df["embedding"].to_numpy())
    X_test  = np.stack(test_df["embedding"].to_numpy())

    sent_clf = SVC(probability=True, random_state=random_state)
    sent_clf.fit(X_train, y_train_sent)
    sentiment_test_acc = sent_clf.score(X_test, y_test_sent)

    test_df["sentiment_score"] = sent_clf.predict_proba(X_test)[:, 1]
    test_df["sentiment_pred"] = (
        (test_df["sentiment_score"] >= 0.5)
        .astype(int)
        .map({0: neg_label, 1: pos_label})
    )

    # --- Gender classifier ---
    train_gender_bin = _gender_to_binary(train_df[gender_col])
    test_gender_bin  = _gender_to_binary(test_df[gender_col])

    gender_mask_train = ~train_gender_bin.isna()
    gender_mask_test  = ~test_gender_bin.isna()

    X_train_g = np.stack(train_df.loc[gender_mask_train, "embedding"].to_numpy())
    y_train_g = train_gender_bin[gender_mask_train].astype(int).to_numpy()

    X_test_g  = np.stack(test_df.loc[gender_mask_test, "embedding"].to_numpy())
    y_test_g  = test_gender_bin[gender_mask_test].astype(int).to_numpy()

    gender_clf = SVC(probability=False, random_state=random_state)
    gender_clf.fit(X_train_g, y_train_g)
    gender_test_acc = gender_clf.score(X_test_g, y_test_g)

    # --- Cosine similarity bias metric ---
    pos_vec = _mean_vec(df[df[sentiment_col].str.lower() == pos_label]["embedding"].to_list())
    neg_vec = _mean_vec(df[df[sentiment_col].str.lower() == neg_label]["embedding"].to_list())
    sent_dir = pos_vec - neg_vec

    female_vec = _mean_vec(df[_gender_to_binary(df[gender_col]) == 1]["embedding"].to_list())
    male_vec   = _mean_vec(df[_gender_to_binary(df[gender_col]) == 0]["embedding"].to_list())
    gender_dir = female_vec - male_vec

    observed_cos = cosine(sent_dir, gender_dir)

    # Permutation test
    perm_values = []
    g_vals = df[gender_col].to_numpy()
    for _ in range(permutation_runs):
        shuffled = g_vals.copy()
        rng.shuffle(shuffled)
        female_vec_perm = _mean_vec(df[_gender_to_binary(pd.Series(shuffled)) == 1]["embedding"].to_list())
        male_vec_perm   = _mean_vec(df[_gender_to_binary(pd.Series(shuffled)) == 0]["embedding"].to_list())
        gender_dir_perm = female_vec_perm - male_vec_perm
        perm_values.append(cosine(sent_dir, gender_dir_perm))

    perm_values = np.array(perm_values, dtype=float)
    p_value = (np.sum(perm_values <= observed_cos) + np.sum(perm_values >= observed_cos)) / (2 * len(perm_values))

    if plot:
        fig = px.histogram(x=perm_values, nbins=40,
                           title="Permutation distribution of cos(sent_dir, gender_dir)")
        fig.add_vline(x=observed_cos)
        fig.show()

    # --- Summaries ---
    counts_by_split = df.groupby([split_col, sentiment_col, gender_col], dropna=False)\
                        .size().rename("count").reset_index()
    test_conf = pd.crosstab(
        index=test_df[sentiment_col].str.lower(),
        columns=test_df["sentiment_pred"].str.lower(),
        dropna=False
    )

    results = {
        "data_with_embeddings": df,
        "train_df": train_df,
        "test_df": test_df,
        "counts_by_split": counts_by_split,
        "sentiment_test_acc": float(sentiment_test_acc),
        "gender_test_acc": float(gender_test_acc),
        "observed_cosine_sent_vs_gender": float(observed_cos),
        "permutation_values": perm_values,
        "permutation_p_value_two_sided": float(p_value),
        "test_confusion_table": test_conf
    }
    return results


# -------- RUN PIPELINE -------- #

results = run_bias_pipeline(
    csv_path="data_set_with_embeddings.csv",
    text_col="text",
    sentiment_col="sentiment",
    gender_col="gender",
    split_col="dataSet",
    embedding_model="text-embedding-3-large",
    permutation_runs=1000,
    plot=True
)

print("Sentiment test accuracy:", results["sentiment_test_acc"])
print("Gender test accuracy:", results["gender_test_acc"])
print("Observed cosine(sent_dir, gender_dir):", results["observed_cosine_sent_vs_gender"])
print("Permutation p-value:", results["permutation_p_value_two_sided"])
